#### Table of Contents
1. [Introduction](#birds-around-my-area-🕊)
2. [Constructing the data pipeline](#constructing-the-data-pipeline)

# Birds Around My Area 🕊
This notebook is an exploratory analysis on bird data provided by Cornell's public eBird API. 

The following steps are taken to construct the data pipeline:
1. list
2. the steps
3. here
4. until done

Pandas and matplotlib are used to further aid in data visualization.

## Constructing the data pipeline

### Get region codes and coordinates using geopy and the FCC Area API
We'll be looking at bird data from Los Angeles.

In order to get the correct regionCode for Los Angeles, we take the following steps:
1. Use the geopy Python library to convert a given location name to it's latidunal and longitdunal coordinates.
2. Use the [FCC Area API](https://geo.fcc.gov/api/census/) to convert coordinates to region codes.
    - Note, eBird region codes follow [ISO 3166-2](https://en.wikipedia.org/wiki/ISO_3166-2#Current_codes) guidelines.

In [141]:
from encodings.utf_8 import getregentry
import requests
import json
from geopy import geocoders

def getLatLong(location):
    """Fetches latitude and longitude of an area.
    
    Args:
        location (str): The location
    Returns:
        tuple: A tuple (lat, long) of the location
        
    """
    geolocator = geocoders.Nominatim(user_agent="Geo Locate")
    location = geolocator.geocode(location)
    
    return ((location.latitude, location.longitude))
    

def getRegionCode(lat, long):
    """Fetches eBird regionCode using municipal census API
    
    Args:
        lat (float): latitude
        long (float): longitude 
    Returns: 
        str: regionCode
    
    """
    
    # Use FCC Area API that is publicly available to get codes for a given coordinate
    censusUrl = str('https://geo.fcc.gov/api/census/area?lat=' + 
                    str(lat) +
                    '&lon=' +
                    str(long) +
                    '&format=json')
    
    # Sending out a GET request
    get = requests.request('GET', censusUrl, data={})
    
    # Parse the response. All API values are contained in the 'results' list
    response = json.loads(get.content)['results'][0]
    
    # regionCode follows ISO 3166-2 guidelines. Each complete code consists of two parts:
    # 1. The ISO 3166-1 alpha-2 code of the country.
    # 2. A string of up to three alphanumeric characters, obtained from already existing codes for countries. Since we'll be looking at bird data in the US, we use the in-state FIPS code.
    fips = response['county_fips']
    regionCode = 'US-' + response['state_code'] + '-' + fips[2] + fips[3] + fips[4]
    
    return regionCode
    
lat_long = getLatLong('Angeles National Forest Los Angeles')
regionCode = getRegionCode(lat_long[0], lat_long[1])

### Connecting to the eBird API to get bird data in JSON format
Note: Python wrapper [ebird-api](https://pypi.org/project/ebird-api/) must be downloaded. Documentation for eBird API and instructions on how to sign up for a key can be found [here](https://documenter.getpostman.com/view/664302/S1ENwy59#4e020bc2-fc67-4fb6-a926-570cedefcc34)

Equipped with our regionCode, we can now search for bird data by region. We can also use function getLatLong() to obtain coordinates for a given region.

In [142]:
from ebird.api import get_observations
import csv
import pandas as pd
import datetime as dt

API_KEY = 'b0e60cbbp1n6'

def getWeeklyBirdData(regionCode):
    """Fetches weekly bird data by calling a GET request from eBird API and converts the request to a CSV
    
    Args:
        regionCode (str): region code for a given location
    Returns:
        DataFrame: DataFrame of the bird data if needed
        
    """
    
    # Get observations from Los Angeles for the past week:
    records = get_observations(API_KEY, regionCode, back=7)
    df = pd.DataFrame(records)
    df.to_csv(getName())
    
    return df
    
def getName():
    """Gets name for bird data file. Has the following structure:
    bird_data + CURRENT_YEAR + WEEK_NUM, where
    WEEK_NUM = the week number of the current day in which the function is being run
    
    Returns:
        String: the data file name
        
    """
    today = dt.date.today()
    week_ago = today - dt.timedelta(days=7)
    
    res = 'bird_data_' + str(week_ago) + '_' + str(today) + '.csv'
    
    return res

dfBird = getWeeklyBirdData(regionCode)


,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,cowscj1,California Scrub-Jay,Aphelocoma californica,L16598980,Wattles Garden Park,2022-08-24 08:37,1.0,34.105140,-118.354776,True,False,False,S117490254
1,caltow,California Towhee,Melozone crissalis,L16598980,Wattles Garden Park,2022-08-24 08:37,2.0,34.105140,-118.354776,True,False,False,S117490254
2,moudov,Mourning Dove,Zenaida macroura,L16598980,Wattles Garden Park,2022-08-24 08:37,1.0,34.105140,-118.354776,True,False,False,S117490254
3,normoc,Northern Mockingbird,Mimus polyglottos,L16598980,Wattles Garden Park,2022-08-24 08:37,4.0,34.105140,-118.354776,True,False,False,S117490254
4,rethaw,Red-tailed Hawk,Buteo jamaicensis,L16598980,Wattles Garden Park,2022-08-24 08:37,1.0,34.105140,-118.354776,True,False,False,S117490254
...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,pomjae,Pomarine Jaeger,Stercorarius pomarinus,L597658,Malibu Lagoon--State Beach including Lagoon,2022-08-19 08:52,2.0,34.033386,-118.682299,True,False,False,S117219689
229,wooduc,Wood Duck,Aix sponsa,L9763951,Los Angeles River -- Marsh Park vic,2022-08-19 08:50,1.0,34.107801,-118.246748,True,False,True,S117224728
230,indbun,Indigo Bunting,Passerina cyanea,L152149,Bette Davis Picnic Area,2022-08-19 08:15,1.0,34.157684,-118.297862,True,True,False,S117216695
231,baleag,Bald Eagle,Haliaeetus leucocephalus,L1608213,San Gabriel Canyon Rd.--below East Fork Rd.,2022-08-19 08:00,1.0,34.217306,-117.857784,True,False,False,S117217594


## Connecting to an existing S3 bucket and storing csv files in them
Using Python library Boto3 we connect to AWS S3. We'll be storing csv files to the *birds-around-my-area* bucket. 

In [80]:
import boto3
import os, glob

def storeToBucket(csvFile, bucket_name):
    """Stores csv file to the specified bucket
    
    Args:
        csvFile (str): name of the csv file
        bucket_name (str): name of the S3 bucket to connect and store data to

    """
    # Creating the connection
    session = boto3.Session(profile_name='default')
    s3 = session.resource('s3')

    # Storing data
    s3_object = s3.Object(bucket_name, csvFile).put(Body=open(csvFile, 'rb'))

    # Delete .csv in local folder
    os.remove(csvFile)
    
    print(csvFile + ' successfully uploaded to ' + bucket_name)
    
storeToBucket(getName(), 'birds-around-my-area')


bird_data_2022-08-16_2022-08-23.csv successfully uploaded to birds-around-my-area


## Creating an Aurora MySQL database using AWS RDS

In [126]:
import pandas as pd
import pymysql

host="ebird-data.cluster-cjhrqfsqr72t.us-west-1.rds.amazonaws.com"
port=3306
dbname="ebirddata"
user="admin"
password="RHgNGm1QyVP0gQG0tItc"

db = pymysql.connect(host=host, user=user,port=port,
                           passwd=password, db=dbname)

cursor = db.cursor()

## Create table birds in the database

In [136]:
import csv

sqlCreateTable = """CREATE TABLE birds (
  indexNum int(11) DEFAULT NULL,
  speciesCode varchar(20) NOT NULL,
  comName varchar(100) DEFAULT NULL,
  sciName varchar(100) DEFAULT NULL,
  locId varchar(20) NOT NULL,
  locName varchar(100) DEFAULT NULL,
  obsDt date DEFAULT NULL,
  howMany int(11) DEFAULT NULL,
  lat double DEFAULT NULL,
  lng double DEFAULT NULL,
  obsValid tinyint(1) DEFAULT NULL,
  obsReviewed tinyint(1) DEFAULT NULL,
  locationPrivate tinyint(1) DEFAULT NULL,
  subId varchar(20) DEFAULT NULL,
  PRIMARY KEY (speciesCode,locId)
);
"""

cursor.execute(sqlCreateTable)

0

## LOAD DATA INTO table using S3

In [139]:
sqlLoadQuery = """LOAD DATA FROM S3 's3://birds-around-my-area/bird_data_2022-08-16_2022-08-23.csv'
INTO TABLE birds
FIELDS TERMINATED BY ',';
"""
cursor.execute(sqlLoadQuery)

IntegrityError: (1062, "Duplicate entry 'specie' for key 'PRIMARY'")

In [ ]:
query1 = """SELECT *
FROM birds
WHERE obsValid=true;
"""

cursor.execute('select * from birds')
print(cursor.fetchall())

In [140]:
db.close()

In [2]:
import inspect

jane = 'Jack'
string = inspect.cleandoc(f"""hello
                          is this {jane}?""")
jane = 'John'
print(string)

hello
is this Jack?


In [13]:
import configparser

config = configparser.ConfigParser()
config_file_path = r'/Users/janekim/Developer/bird proj/birds/config.ini'
config.read(config_file_path)
API_KEY = config.get('ebird', 'API_KEY')

config.sections()
API_KEY

'b0e60cbbp1n6'

In [15]:
bucket_name = 'HI'
filename = 'HEY'

load_query = inspect.cleandoc(f"""--sql
                                    LOAD DATA FROM S3 's3://{bucket_name}/{filename}'
                                    INTO TABLE birds
                                    FIELDS TERMINATED BY ',';""")

print(load_query)

--sql
LOAD DATA FROM S3 's3://HI/HEY'
INTO TABLE birds
FIELDS TERMINATED BY ',';


In [16]:
import pandas as pd

df = pd.read_csv('bird_data_2022-08-27.csv')
df.head()

,Unnamed: 0,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,19,bknsti,Black-necked Stilt,Himantopus mexicanus,L152149,Bette Davis Picnic Area,2022-08-27 18:39,200,34.157684,-118.297862,True,False,False,S117678011
1,106,cangoo,Canada Goose,Branta canadensis,L467098,Whittier Narrows Rec Area,2022-08-27 14:40,120,34.039927,-118.063154,True,False,False,S117674788
2,87,lobdow,Long-billed Dowitcher,Limnodromus scolopaceus,L13730093,Los Angeles River--Anaheim St to Golden Shore,2022-08-27 16:05,110,33.769404,-118.204914,True,False,False,S117680619
3,8,recpar,Red-crowned Parrot,Amazona viridigenalis,L20695685,"780–784 S Arroyo Pkwy, Pasadena US-CA 34.13242...",2022-08-27 19:14,47,34.132418,-118.147260,True,False,True,S117678067
4,10,mallar3,Mallard,Anas platyrhynchos,L152149,Bette Davis Picnic Area,2022-08-27 18:39,45,34.157684,-118.297862,True,False,False,S117678011
